In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.tensorboard import SummaryWriter
from google.colab import drive



In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Define the dataset path
dataset_path = "/content/drive/MyDrive/Vegetable/Images/Vegetable"



In [ ]:
dataset_path

'/content/drive/MyDrive/Vegetable/Images/Vegetable'

In [1]:
##########################################

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import ImageFolder
import torchvision.models as models
import albumentations as A

# Set up distributed training environment if needed

# Define transformations using albumentations library
from torchvision.transforms import ToTensor, Resize
train_transform = transforms.Compose([
    Resize((224, 224)),  
    ToTensor(),  
    
])

val_transform = transforms.Compose([
    Resize((224, 224)),  
    ToTensor(),  
   
])

test_transform = transforms.Compose([
    Resize((224, 224)),  
    ToTensor(),  
   
])


In [14]:
# Define dataset paths
train_dataset = ImageFolder(dataset_path + '/train', transform=train_transform)
test_dataset = ImageFolder(dataset_path + '/test', transform=test_transform)
val_dataset = ImageFolder(dataset_path + '/validation', transform=val_transform)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True)
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set up TensorBoard writer for logging
writer = SummaryWriter('logs')



In [16]:
# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100.0 * train_correct / train_total
    train_loss /= len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100.0 * val_correct / val_total
    val_loss /= len(val_loader)

    # Log metrics to TensorBoard
    writer.add_scalar('Loss/Train', train_loss, epoch)
    writer.add_scalar('Loss/Val', val_loss, epoch)
    writer.add_scalar('Accuracy/Train', train_accuracy, epoch)
    writer.add_scalar('Accuracy/Val', val_accuracy, epoch)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Train Acc: {train_accuracy:.2f}% | Val Acc: {val_accuracy:.2f}%")

# Save the trained model
torch.save(model.state_dict(), 'vegetable_classifier.pth')

# Close the TensorBoard writer
writer.close()

Epoch 1/100 | Train Loss: 0.7838 | Val Loss: 3.4162 | Train Acc: 86.44% | Val Acc: 51.11%
Epoch 2/100 | Train Loss: 0.6919 | Val Loss: 2.7435 | Train Acc: 84.89% | Val Acc: 52.44%
Epoch 3/100 | Train Loss: 0.7257 | Val Loss: 4.0579 | Train Acc: 88.89% | Val Acc: 51.56%
Epoch 4/100 | Train Loss: 1.0443 | Val Loss: 6.6931 | Train Acc: 89.78% | Val Acc: 42.67%
Epoch 5/100 | Train Loss: 0.7591 | Val Loss: 2.0336 | Train Acc: 90.00% | Val Acc: 65.33%
Epoch 6/100 | Train Loss: 0.9671 | Val Loss: 3.0871 | Train Acc: 83.33% | Val Acc: 52.89%
Epoch 7/100 | Train Loss: 0.8610 | Val Loss: 1.7872 | Train Acc: 86.89% | Val Acc: 67.11%
Epoch 8/100 | Train Loss: 0.4902 | Val Loss: 2.6103 | Train Acc: 87.56% | Val Acc: 64.89%
Epoch 9/100 | Train Loss: 0.6497 | Val Loss: 1.8274 | Train Acc: 88.22% | Val Acc: 68.00%
Epoch 10/100 | Train Loss: 0.8631 | Val Loss: 5.1362 | Train Acc: 89.11% | Val Acc: 50.67%
Epoch 11/100 | Train Loss: 0.8757 | Val Loss: 0.9542 | Train Acc: 92.44% | Val Acc: 78.67%
Epoch 12

In [17]:
# Validating

In [33]:
from PIL import Image

# Load and preprocess the image
image_path = '/content/drive/MyDrive/Vegetable/Images/Vegetable/validation/Bean/1222.jpg'
image = Image.open(image_path).convert('RGB')
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_image = transform(image).unsqueeze(0).to(device)


In [34]:
with torch.no_grad():
    output = model(input_image)
    _, predicted = torch.max(output.data, 1)

predicted_label = train_dataset.classes[predicted.item()]
print("Predicted label:", predicted_label)


Predicted label: Bean
